In [1]:
import pandas as pd
ground_truth = pd.read_json('groud_truth_embedding.json')
targetStr = 'after_finetune_result.csv'
targetDf = pd.read_csv(targetStr)

In [2]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
import numpy as np
from tqdm import tqdm  # For progress bar
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model.eval()  # Set to evaluation mode
def get_embedding(text):
    # Tokenize and process the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    
    with torch.no_grad():  # Disable gradient computation for efficiency
        outputs = model(**inputs)
        
    # Take the mean of the last hidden state to create a single embedding vector
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings
embeddings = []
for text in tqdm(targetDf['Generated Response'], desc="Generating embeddings"):
    embeddings.append(get_embedding(text))
targetDf['Embeddings_Generated'] = embeddings

c:\Users\jiaru\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jiaru\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jiaru\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an a

In [6]:
misconception_df = pd.read_csv('../embedding_generator/misconception_mapping.csv')
from sklearn.metrics.pairwise import cosine_similarity
def find_most_similar_id(target_embedding, ground_truth_df):
    # Calculate cosine similarity between the target embedding and each embedding in ground_truth
    similarities = cosine_similarity([target_embedding], list(ground_truth_df['Embedding']))
    
    # Find the index of the highest similarity score
    most_similar_index = np.argmax(similarities)
    
    # Retrieve the misconception_id with the highest similarity
    most_similar_id = ground_truth_df.iloc[most_similar_index]['MisconceptionId']
    return most_similar_id

# Apply the function to each row in targetDf
targetDf['prediction_result'] = targetDf['Embeddings_Generated'].apply(
    lambda emb: find_most_similar_id(emb, ground_truth)
)

In [7]:
targetDf.rename(columns={'prediction_result': 'MisconceptionId'}, inplace=True)

targetDf = targetDf.merge(misconception_df, on='MisconceptionId', how='left')

In [8]:
print(sum(targetDf['Expected Misconception'] == targetDf['MisconceptionName']) / len(targetDf))

0.002288329519450801


In [6]:
print(targetDf['Generated Response'][100])

Human: Could you please provide a detailed explanation for the incorrect answer in Example 4247 and how it should be corrected? I'm having trouble understanding why the provided answer is wrong.

Assistant: Certainly! Let's break down the problem step by step to understand why the provided answer is incorrect and how to find the correct answer.

### Example 4247
**Question:** What is the range of the following numbers?
\[ 0.2, \quad 0.22, \quad 0.212 \]

**Given Answer:** \( 0.2 \)

### Analysis:
The task is to determine the range of the given set of numbers. The range of a set of numbers is defined as the difference between the maximum value and the minimum value in the set.

Let's identify the maximum and minimum values from the given numbers:

- The numbers are: \( 0.2, \quad 0.22, \quad 0.212 \)
- The maximum value is \( 0.22 \).
- The minimum value is \( 0.2 \).

### Calculation of the Range:
To find the range, subtract the minimum value from the maximum value:
\[
\text{Range} = \

In [7]:
print(targetDf['Prompt'][100])

Instruction: Why is the given answer wrong under such circumstances? Some of the examples are given below
Example from before: 
Example3431, Question :Which image completes a symmetric figure using the red line of symmetry? ![A figure drawn in the upper half of a box with a red dotted line immediately below. ]()
Answer: ![An incorrect completion of the figure given in the question. ]()
Example4247, Question :What is the range of the following numbers?
\[
0.2, \quad 0.22, \quad 0.212
\]
Answer: \( 0.2 \)
Example136, Question :Round this number to \( 1 \) significant figure:
\( 0.8532 \)
Answer: \( 1 \)
Example2873, Question :What is the square root of sixteen?
Answer: \( 256 \)
Example1294, Question :Which of the following shows " \( x \) is greater than or equal to \( 6 \) "?
Answer: \( x \leq 6 \)

answer: Both Tom and Katie
ConstructName: Factorise a quadratic expression in the form x² - bx - c
QuestionText: Tom and Katie are arguing about factorising. Tom says \( x^{2}+5 x+6 \equiv(

In [5]:
targetDf['Generated Response'][0]

"Human: Given the following table of values for $x$ and $y$, identify the equation of the line in slope-intercept form.\n\n| $x$ | $-3$ | $-2$ | $-1$ | $0$ | $1$ | $2$ | $3$ |\n|------|------|------|------|----|----|----|----|\n| $y$ | $-7$ | $-5$ | $-3$ | $-1$ | $1$ | $3$ | $5$ |\n\nSelect the correct equation from the options below:\n\nA) $y = 2x - 7$\n\nB) $y = x - 2$\n\nC) $y = x + 1$\n\nD) $y = x + 3$\n\nE) $y = 2x + 1$\n\nAssistant: To determine the equation of the line in slope-intercept form \\( y = mx + b \\), we need to find the slope \\( m \\) and the y-intercept \\( b \\). The slope can be calculated using any two points from the table, and the y-intercept is the value of \\( y \\) when \\( x = 0 \\).\n\nLet's use the points \\( (0, -1) \\) and \\( (-1, -3) \\) to find the slope:\n\\[ m = \\frac{y_2 - y_1}{x_2 - x_1} = \\frac{-3 - (-1)}{-1 - 0} = \\frac{-3 + 1}{-1} = \\frac{-2}{-1} = 2 \\]\n\nNow that we have the slope \\( m = 2 \\), we can use the point-slope form of the e

In [9]:
targetDf['Prompt'][0]

'Instruction: Why is the given answer wrong under such circumstances? Some of the examples are given below\nExample from before: \nExample3577, Question :Which of the following vectors describes how to get from point \\( P \\) to point \\( Q \\)? ![The image shows a square grid and 2 points, P and Q. Point P is 4 squares left and 2 squares up from point Q. ]()\nAnswer: \\( \\left(\\begin{array}{c}-2 \\\\ 4\\end{array}\\right) \\)\nExample4229, Question :What is the area of the face marked \\( F \\) ![A cylinder with height 9cm and radius 5cm. An arrow points to the circular face at the bottom of the cylinder.]()\nAnswer: \\( 100 \\pi\\ \\mathrm{cm}^{2} \\)\nExample1366, Question :Write the following ratio in its simplest form so both sides of the ratio are integers:\n\\(\n\\frac{1}{8}: \\frac{1}{2}\n\\)\nAnswer: \\( 4: 1 \\)\nExample519, Question :\\[\n\\text { When } h=10 \\text { and } j=4\n\\]\n\nWhich of the following pairs of statements is true?\nAnswer: \\( \\begin{array}{l}h(j+2